<a href="https://colab.research.google.com/github/Yewon9/STT_JEJU/blob/main/Wav2Vec2_Baseline_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
!pip install transformers datasets torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00


In [8]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torchaudio
from transformers import Trainer, TrainingArguments
import json
import matplotlib.pyplot as plt

# tokenizer & model

In [4]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

# DATA

In [6]:
data = pd.read_csv('sampled_data.csv')
data

,Path,Text,Standard Text,Audio_Length
0,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,아 난 다른 운동은 안 허고 그냥 걷는 것이 다리 아직은 안 아프난 기냥 걸어그네 ...,아 난 다른 운동은 안 하고 그냥 걷는 것이 다리 아직은 안 아프니까 그냥 걸어서 ...,10.000
1,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,큰아이도 이제 공기업에 있고 우리 작은애는 이제 공무원 되고 하나 결혼해서 이제 애...,큰아이도 이제 공기업에 있고 우리 작은애는 이제 공무원 되고 하나 결혼해서 이제 아...,24.000
2,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,아침에 일어나면 딱 눈 뜨면 커피 먹고 커피 먹고 난 다음에 유튜브 유튜브에서 이것...,아침에 일어나면 딱 눈 뜨면 커피 먹고 커피 먹고 난 다음에 유튜브 유튜브에서 이것...,13.930
3,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,젊을 때는 길에서 넘어지면 그냥 거기서 약 바르면 되지 뭐 할 수 있어요 그래도 되...,젊을 때는 길에서 넘어지면 그냥 거기서 약 바르면 되지 뭐 할 수 있어요 그래도 되...,26.690
4,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,나가 그자 온 것이 아니고 이 집이 큰 똘을 중신허젠 왔수다,내가 그냥 온 것이 아니고 이 집이 큰 딸을 중신하려고 왔습니다,6.299
...,...,...,...,...
5995,/content/drive/MyDrive/JEJU/data/wav/TL_02/say...,재작년에 성안에 이신 한국병원이라는 건디가 받았신디 건강검진센터에 강인해 이 검사를...,재작년에 성안에 있는 한국병원이라는 건디가 받았는데 건강검진센터에 강인해 이 검사를...,18.959
5996,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,느네 아바이영 어멍이영 혼디 어디 감신고이,너희 아버지하고 어머니하고 함께 어디 가시고 있지,4.699
5997,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,똠떼기사 시원허민 나스난 거씬 모욕헤 불라,땀띠는 시원하면 나으니 얼른 목욕해 버려라,7.280
5998,/content/drive/MyDrive/JEJU/data/wav/VL_01/st_...,곹이 음식을 먹어신디 누긴 두드러기가 막 나고 누긴 펜지롱헷저이,같이 음식을 먹었는데 누구는 두드러기가 막 나고 누구는 말짱했죠,6.260


In [7]:
dataset = Dataset.from_pandas(data)

# 음성 파일 처리 함수

In [9]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["Path"])
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [ ]:
dataset = dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

# 토크나이저 적용

In [ ]:
def tokenize_text(batch):
    batch["input_values"] = tokenizer(batch["speech"], return_tensors="pt", padding="longest").input_values
    batch["labels"] = tokenizer(batch["Text"], return_tensors="pt", padding="longest").input_ids
    return batch

In [ ]:
dataset = dataset.map(tokenize_text, remove_columns=["speech", "Text"])

# 데이터셋 분리

In [ ]:
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

In [ ]:
dataset_dict = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['test'],
    'test': test_valid['train']
})

# 모델 학습 설정

In [ ]:
training_args = TrainingArguments(
  output_dir="./wav2vec2-basic",
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  weight_decay=0.01,
  logging_dir='./logs',
  logging_steps=10,
  save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=tokenizer,
)

# 모델 학습

In [ ]:
trainer.train()

# 모델 성능 평가

In [ ]:
results = trainer.evaluate(dataset_dict["test"])

print(f"Test Results: {results}")

# 모델 결과 저장

In [ ]:
model.save_pretrained('./wav2vec2-basic-model')
tokenizer.save_pretrained('./wav2vec2-basic-tokenizer')

In [ ]:
with open('results.json', 'w') as f:
    json.dump(results, f)

# 손실값 시각화

In [ ]:
loss_values = trainer.state.log_history

In [ ]:
epochs = range(1, len(loss_values)+1)
plt.plot(epochs, loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()